In [ ]:
!git init
!git config --global user.email 'ylgiraldoe@uqvirtual.edu.co'
!git config --global user.name 'Yeicko10'
from getpass import getpass
password = getpass('1094923518Yeicko')
!git clone https://github.com/Yeicko10/Aprendiendo-git-proyecto1/blob/main/Parcial_1_Analisis_Exploratorio_De_Datos.ipynb
%cd Parcial_1_Analisis_Exploratorio_De_Datos.ipynb
# create a file, then add it to stage
!git commit -m 'Subimos base de datos desde drive'  # commit in Colab
!git push origin master          # push to github'

Reinitialized existing Git repository in /content/.git/
1094923518Yeicko··········
Cloning into 'Parcial_1_Analisis_Exploratorio_De_Datos.ipynb'...
fatal: repository 'https://github.com/Yeicko10/Aprendiendo-git-proyecto1/blob/main/Parcial_1_Analisis_Exploratorio_De_Datos.ipynb/' not found
[Errno 2] No such file or directory: 'Parcial_1_Analisis_Exploratorio_De_Datos.ipynb'
/content
On branch master

Initial commit

Untracked files:
	.config/
	drive/
	sample_data/

nothing added to commit but untracked files present
error: src refspec master does not match any.
error: failed to push some refs to 'origin'


In [ ]:
dirBase = '/MyDrive/Datos_Tesis_Yeicko/'
!pwd

/content


In [ ]:
import numpy as np
import pandas as pd


import matplotlib.pyplot as plt

import scipy.ndimage
from numpy import nan
import time
import sys
import random

import gc

import shelve

import os
from shutil import copyfile


In [ ]:
# Determinamos la plataforma
if sys.platform == 'linux':
    print("Entorno Linux - Google Colab")
    
    # Utilizacion de TPU (Referencia: https://colab.research.google.com/notebooks/tpu.ipynb)
    
    # Determinamos si tenemos disponible en el entorno una GPU, una TPU o bien solo CPU
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
        print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
        entorno = 'TPU'
    except ValueError:

        #if tf.test.is_gpu_available():
        if len(tf.config.list_physical_devices('GPU')) > 0:
            entorno = 'GPU'
        else:
            entorno = 'CPU'
        
    if entorno == 'TPU':
        # Configuración para uso de TPU
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

    prefijo = '/content/drive'
else:
    print("Entorno Windows - Anaconda")
    prefijo = 'C:/Users/jandro/DriveFormacion'
    entorno = 'CPU'

dirData = prefijo + dirBase + "DATASET/"
dirResultados = prefijo + dirBase + "RESULTS/"

print("Ejecutando en entorno:",entorno)

Entorno Linux - Google Colab
Ejecutando en entorno: CPU


In [ ]:
# Recuperamos conjunto de datos
if sys.platform == 'linux':

    # Estamos en Google Colab
    from google.colab import drive
    drive.mount('/content/drive')
    ##modo + '_preprocessed_test.csv')
    dfK2 = pd.read_csv(dirData + 'exoplanets.csv')
     
else:
    dfK2 = pd.read_csv(dirData + 'exoplanets.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (4,5,10,11,17,18,23,24,26,27,30,33,38,39,44,45,50,51,56,57,62,63,70,71,73,74,79,80,81,82,88,89,95,96,105,106,109,110,115,116,117,124,125,130,131,137,143,144,149,150,159,160,161,162,172,173,174,179,180,182,188,189,195,196,201,202,205,206,211,212,217,218,223,224,229,230,235,236,241,242,247,248,253,254,259,260,265,266,268,269,270,271,273,278,279,284,285,290,291,294,295,301,302,304,305,310,311) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
dfK2.head()

,A,AUPPER,ALOWER,UA,AREF,AURL,AR,ARUPPER,ARLOWER,UAR,...,VREF,VURL,VSINI,VSINIUPPER,VSINILOWER,UVSINI,VSINIREF,VSINIURL,KEPID,KDE
0,0.078010,0.001300,0.001300,0.001300,Calculated,http://adsabs.harvard.edu/abs/2006ApJ...646..505B,11.9153,0.443805,0.443805,0.443805,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,0.034472,0.000676,0.000676,0.000676,Calculated,http://adsabs.harvard.edu/abs/2006ApJ...646..505B,15.1619,0.974724,0.974724,0.974724,...,NaN,NaN,NaN,NaN,NaN,NaN,Morton 2016,http://adsabs.harvard.edu/abs/2016ApJ...822...86M,NaN,1
2,0.137610,0.002897,0.002897,0.002897,Calculated,http://adsabs.harvard.edu/abs/2006ApJ...646..505B,31.8896,3.494090,3.494090,3.494090,...,NaN,NaN,NaN,NaN,NaN,NaN,Morton 2016,http://adsabs.harvard.edu/abs/2016ApJ...822...86M,NaN,1
3,0.091351,0.001903,0.001903,0.001903,Calculated,http://adsabs.harvard.edu/abs/2006ApJ...646..505B,14.5835,2.181770,2.181770,2.181770,...,NaN,NaN,3.0,1.0,1.0,1.0,Hebrard 2014,http://adsabs.harvard.edu/abs/2014A%26A...572A...,NaN,1
4,0.185149,0.008817,0.008817,0.008817,Calculated,http://adsabs.harvard.edu/abs/2006ApJ...646..505B,33.2523,10.100600,10.100600,10.100600,...,NaN,NaN,NaN,NaN,NaN,NaN,Morton 2016,http://adsabs.harvard.edu/abs/2016ApJ...822...86M,NaN,1


In [ ]:
dfK2.shape

(5747, 314)